In [59]:
import pyarrow as pa

In [60]:
hdfs = pa.hdfs.HadoopFileSystem(host="10.3.0.2", port=8020)

In [61]:
hdfs.ls('/user/swapanc1')

['/user/swapanc1/homework1_images']

## Downloading the Csv Files using HDFS.

In [62]:
hdfs.download('/data/google_open_image/ids/test-images-with-rotation.csv', stream = 'test-images-with-rotation.csv', buffer_size=None)
hdfs.download('/data/google_open_image/labels/test-annotations-human-imagelabels-boxable.csv', stream = 'test-annotations-human-imagelabels-boxable.csv', buffer_size=None)
hdfs.download('/data/google_open_image/metadata/class-descriptions-boxable.csv', stream = 'class-descriptions-boxable.csv', buffer_size=None)

## Step 2: Creating separate Dataframes for all the 3 CSV Files.

In [63]:
from pyarrow import csv
import pandas as pd
file1 = 'test-images-with-rotation.csv'
file2='test-annotations-human-imagelabels-boxable(1).csv'
file3='class-descriptions-boxable.csv'
df1=pd.read_csv(file1)
df2=pd.read_csv(file2)
df3=pd.read_csv(file3)

In [64]:
df1.head(2)

,ImageID,Subset,OriginalURL,OriginalLandingURL,License,AuthorProfileURL,Author,Title,OriginalSize,OriginalMD5,Thumbnail300KURL,Rotation
0,848d938b566f5e9c,test,https://c5.staticflickr.com/1/125/321313101_ba...,https://www.flickr.com/photos/calliope/321313101,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/calliope/,liz west,Tonka2,2968041,t8MQie4TFc6v4xFhzclUZA==,https://c4.staticflickr.com/1/125/321313101_ba...,0.0
1,dab3e432f1ee4ae4,test,https://farm5.staticflickr.com/8790/1723529903...,https://www.flickr.com/photos/92414546@N04/172...,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/92414546@N04/,davejdoe,Goals,9146892,SE2ABMxpBmqDaq+dan13UA==,https://c1.staticflickr.com/9/8790/17235299036...,0.0


In [65]:
df1.shape

(125436, 12)

In [66]:
df2.head(2)

,ImageID,Source,LabelName,Confidence
0,000026e7ee790996,verification,/m/0cgh4,0
1,000026e7ee790996,verification,/m/04hgtk,0


In [67]:
df2.shape

(772776, 4)

In [68]:
df3.head(2)

,/m/011k07,Tortoise
0,/m/011q46kg,Container
1,/m/012074,Magpie


## The 3rd CSV File contains No column Name, hence adding Column Names called Label and Item and the adding the deleted value back to the Data Frame.

In [69]:
df3.columns =['Label', 'Item']
df3.head(2)

,Label,Item
0,/m/011q46kg,Container
1,/m/012074,Magpie


In [70]:
df3.loc['0',:]=['/m/011k07','Tortoise']  
df3.tail(2)

,Label,Item
599,/m/0zvk5,Helmet
0,/m/011k07,Tortoise


In [71]:
df3.shape

(601, 2)

## Use the join command to join the three dataframes into a single dataframe. Ensure you don't have repeated lines in the resulting dataframe. File 3 can be joined with file 2 based on the Label field. (File 2 calls this LabelName, File 3 calls it Label) That dataframe can be joined with File 1 based on the Image ID field.

In [72]:
join_df = df2.join(df3.set_index('Label'), on='LabelName')
join_df.head(15)

,ImageID,Source,LabelName,Confidence,Item
0,000026e7ee790996,verification,/m/0cgh4,0,Building
1,000026e7ee790996,verification,/m/04hgtk,0,Human head
2,000026e7ee790996,verification,/m/0d5gx,0,Castle
3,000026e7ee790996,verification,/m/07j7r,1,Tree
4,000026e7ee790996,verification,/m/04rky,0,Mammal
5,000026e7ee790996,verification,/m/05s2s,1,Plant
6,000062a39995e348,verification,/m/05n4y,0,Ostrich
7,000062a39995e348,verification,/m/04rky,0,Mammal
8,000062a39995e348,verification,/m/07j7r,0,Tree
9,000062a39995e348,verification,/m/015p6,1,Bird


In [73]:
join_df.shape

(772776, 5)

In [74]:
join_df.tail()

,ImageID,Source,LabelName,Confidence,Item
772771,fffc6543b32da1dd,verification,/m/07yv9,0,Vehicle
772772,fffc6543b32da1dd,verification,/m/0cnyhnx,1,Bull
772773,fffc6543b32da1dd,verification,/m/0jbk,1,Animal
772774,fffcedcc4cce3c9f,verification,/m/09j2d,0,Clothing
772775,fffd0258c243bbea,verification,/m/01g317,1,Person


In [75]:
join_df2=join_df.join(df1.set_index('ImageID'), on='ImageID')
join_df2.head(2)

,ImageID,Source,LabelName,Confidence,Item,Subset,OriginalURL,OriginalLandingURL,License,AuthorProfileURL,Author,Title,OriginalSize,OriginalMD5,Thumbnail300KURL,Rotation
0,000026e7ee790996,verification,/m/0cgh4,0,Building,test,https://farm7.staticflickr.com/5769/2109480371...,https://www.flickr.com/photos/132646954@N02/21...,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/132646954@N02/,dronepicr,Chichen Itza Mexico,7825604,EEGQ0ZQ/OxJAIGzfJ6LMvw==,https://c1.staticflickr.com/6/5769/21094803716...,0.0
1,000026e7ee790996,verification,/m/04hgtk,0,Human head,test,https://farm7.staticflickr.com/5769/2109480371...,https://www.flickr.com/photos/132646954@N02/21...,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/132646954@N02/,dronepicr,Chichen Itza Mexico,7825604,EEGQ0ZQ/OxJAIGzfJ6LMvw==,https://c1.staticflickr.com/6/5769/21094803716...,0.0


In [76]:
join_df2.shape

(772776, 16)

In [77]:
join_df2.iloc[0,6]

'https://farm7.staticflickr.com/5769/21094803716_da3cea21b8_o.jpg'

In [78]:
join_df2.head()

,ImageID,Source,LabelName,Confidence,Item,Subset,OriginalURL,OriginalLandingURL,License,AuthorProfileURL,Author,Title,OriginalSize,OriginalMD5,Thumbnail300KURL,Rotation
0,000026e7ee790996,verification,/m/0cgh4,0,Building,test,https://farm7.staticflickr.com/5769/2109480371...,https://www.flickr.com/photos/132646954@N02/21...,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/132646954@N02/,dronepicr,Chichen Itza Mexico,7825604,EEGQ0ZQ/OxJAIGzfJ6LMvw==,https://c1.staticflickr.com/6/5769/21094803716...,0.0
1,000026e7ee790996,verification,/m/04hgtk,0,Human head,test,https://farm7.staticflickr.com/5769/2109480371...,https://www.flickr.com/photos/132646954@N02/21...,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/132646954@N02/,dronepicr,Chichen Itza Mexico,7825604,EEGQ0ZQ/OxJAIGzfJ6LMvw==,https://c1.staticflickr.com/6/5769/21094803716...,0.0
2,000026e7ee790996,verification,/m/0d5gx,0,Castle,test,https://farm7.staticflickr.com/5769/2109480371...,https://www.flickr.com/photos/132646954@N02/21...,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/132646954@N02/,dronepicr,Chichen Itza Mexico,7825604,EEGQ0ZQ/OxJAIGzfJ6LMvw==,https://c1.staticflickr.com/6/5769/21094803716...,0.0
3,000026e7ee790996,verification,/m/07j7r,1,Tree,test,https://farm7.staticflickr.com/5769/2109480371...,https://www.flickr.com/photos/132646954@N02/21...,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/132646954@N02/,dronepicr,Chichen Itza Mexico,7825604,EEGQ0ZQ/OxJAIGzfJ6LMvw==,https://c1.staticflickr.com/6/5769/21094803716...,0.0
4,000026e7ee790996,verification,/m/04rky,0,Mammal,test,https://farm7.staticflickr.com/5769/2109480371...,https://www.flickr.com/photos/132646954@N02/21...,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/132646954@N02/,dronepicr,Chichen Itza Mexico,7825604,EEGQ0ZQ/OxJAIGzfJ6LMvw==,https://c1.staticflickr.com/6/5769/21094803716...,0.0


## Step 3: Download Images

In [79]:
import requests
response = requests. get(join_df2.iloc[5,6])
file = open("image.jpg", "wb")
file. write(response. content)
file.close()

In [80]:
response = requests. get(join_df2.iloc[7,6])
file = open("image1.jpg", "wb")
file. write(response. content)
file.close()

In [81]:
response = requests. get(join_df2.iloc[16,6])
file = open("image2.jpg", "wb")
file. write(response. content)
file.close()

## Creaing a New Directory using mkdir

In [82]:
!hdfs dfs -mkdir /user/swapanc1/homework1_images

mkdir: `/user/swapanc1/homework1_images': File exists


## Step 4: Uploading the downloaded Images to HDFS

In [83]:
!hadoop fs -copyFromLocal 'image.jpg' '/user/swapanc1/homework1_images'
!hadoop fs -copyFromLocal 'image1.jpg' '/user/swapanc1/homework1_images'
!hadoop fs -copyFromLocal 'image2.jpg' '/user/swapanc1/homework1_images'

copyFromLocal: `/user/swapanc1/homework1_images/image.jpg': File exists
copyFromLocal: `/user/swapanc1/homework1_images/image1.jpg': File exists
copyFromLocal: `/user/swapanc1/homework1_images/image2.jpg': File exists


In [84]:
hdfs.ls('/user/swapanc1/homework1_images') ## Checking the uploaded files in the directory.

['/user/swapanc1/homework1_images/image.jpg',
 '/user/swapanc1/homework1_images/image1.jpg',
 '/user/swapanc1/homework1_images/image2.jpg']